In [ ]:
# Import libraries
import duckdb
import json
import pandas as pd
import pyarrow.dataset as ds
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
from pathlib import Path
from datetime import datetime

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11

# Data directory
DATA_DIR = Path(r"c:\Users\User\Desktop\VibeCoding\PolyMarketScrapping\data")

print(f"Data directory: {DATA_DIR}")
print(f"Run time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Data Discovery

Check what data files are available.

In [ ]:
# Count available data
data_stats = {}
for folder in ['leaderboard', 'trades', 'gamma_markets', 'prices']:
    folder_path = DATA_DIR / folder
    if folder_path.exists():
        parquet_count = sum(1 for _ in folder_path.rglob('*.parquet'))
        data_stats[folder] = parquet_count
        print(f"   {folder}: {parquet_count} parquet files")
    else:
        data_stats[folder] = 0
        print(f"   {folder}: NOT FOUND")

## 2. Load Market Resolution Data

Parse the gamma_markets data to determine which markets have resolved and their final prices.

In [ ]:
def safe_json_parse(s, idx):
    """Safely parse JSON array and extract element at index."""
    if pd.isna(s) or s == '' or s == '[]':
        return None
    try:
        arr = json.loads(s)
        if isinstance(arr, list) and len(arr) > idx:
            return arr[idx]
        return None
    except:
        return None


def safe_json_float(s, idx):
    """Safely parse JSON array and extract float at index."""
    val = safe_json_parse(s, idx)
    if val is not None:
        try:
            return float(val)
        except:
            return None
    return None


# Load gamma markets
print("Loading gamma markets...")
gamma_path = DATA_DIR / "gamma_markets"
dataset = ds.dataset(gamma_path, partitioning="hive")
markets_df = dataset.to_table().to_pandas()

print(f"Loaded {len(markets_df):,} market records")

# Parse JSON columns safely
print("Parsing outcome data...")
markets_df['outcome1_name'] = markets_df['outcomes'].apply(lambda x: safe_json_parse(x, 0))
markets_df['outcome2_name'] = markets_df['outcomes'].apply(lambda x: safe_json_parse(x, 1))
markets_df['outcome1_final_price'] = markets_df['outcomePrices'].apply(lambda x: safe_json_float(x, 0))
markets_df['outcome2_final_price'] = markets_df['outcomePrices'].apply(lambda x: safe_json_float(x, 1))

# Mark resolved markets (price is exactly 0 or 1)
markets_df['is_resolved'] = (
    (markets_df['closed'] == True) & 
    (
        (markets_df['outcome1_final_price'].isin([0.0, 1.0])) | 
        (markets_df['outcome2_final_price'].isin([0.0, 1.0]))
    )
)

print(f"Found {markets_df['is_resolved'].sum():,} resolved markets")

market_resolution = markets_df[['conditionId', 'question', 'category', 'closed', 
                   'outcome1_name', 'outcome2_name', 
                   'outcome1_final_price', 'outcome2_final_price', 'is_resolved']]

market_resolution.head()

## 3. Create DuckDB Views

Set up DuckDB connection and create views for leaderboard and trades data.

In [ ]:
# Create DuckDB connection
con = duckdb.connect(':memory:')

# Register market resolution as view
con.register('market_resolution', market_resolution)

# Create leaderboard view
con.execute(f"""
    CREATE VIEW v_leaderboard AS
    SELECT 
        TRY_CAST(rank AS INTEGER) as rank_num,
        proxyWallet,
        userName,
        xUsername,
        vol,
        pnl,
        dt
    FROM read_parquet('{DATA_DIR}/leaderboard/**/*.parquet', 
                      hive_partitioning=true, union_by_name=true)
""")

# Create trades view
con.execute(f"""
    CREATE VIEW v_trades AS
    SELECT 
        proxyWallet,
        side,
        price,
        size,
        conditionId,
        timestamp,
        outcome,
        price * size AS trade_value_usd
    FROM read_parquet('{DATA_DIR}/trades/**/*.parquet',
                      hive_partitioning=true, union_by_name=true)
""")

print("DuckDB views created successfully!")

## 4. Identify Top 500 Users

Get the top 500 users from the leaderboard based on their PnL.

In [ ]:
# Configuration
TOP_N = 500  # Change this to copy fewer/more users

# Get top N users
top_users_df = con.execute(f"""
    WITH latest_leaderboard AS (
        SELECT 
            proxyWallet,
            userName,
            pnl,
            vol,
            dt,
            ROW_NUMBER() OVER (PARTITION BY proxyWallet ORDER BY dt DESC) as rn
        FROM v_leaderboard
        WHERE rank_num <= {TOP_N}
    )
    SELECT DISTINCT
        proxyWallet,
        userName,
        pnl as latest_pnl,
        vol as total_volume
    FROM latest_leaderboard
    WHERE rn = 1
    ORDER BY latest_pnl DESC
    LIMIT {TOP_N}
""").fetchdf()

# Register top users
con.register('top_users', top_users_df)

print(f"Found {len(top_users_df)} top users")
print(f"Combined PnL: ${top_users_df['latest_pnl'].sum():,.2f}")
print(f"Combined Volume: ${top_users_df['total_volume'].sum():,.2f}")

# Show top 10
top_users_df.head(10)

## 5. Get Trades from Top Users

Fetch all trades made by the top users.

In [ ]:
# Get trade summary
trades_summary = con.execute("""
    SELECT 
        COUNT(*) as total_trades,
        COUNT(DISTINCT t.proxyWallet) as active_traders,
        COUNT(DISTINCT t.conditionId) as unique_markets,
        SUM(t.trade_value_usd) as total_volume,
        MIN(to_timestamp(t.timestamp)) as earliest_trade,
        MAX(to_timestamp(t.timestamp)) as latest_trade
    FROM v_trades t
    INNER JOIN top_users u ON t.proxyWallet = u.proxyWallet
""").fetchone()

print(f"Total trades: {trades_summary[0]:,}")
print(f"Active traders: {trades_summary[1]:,}")
print(f"Unique markets: {trades_summary[2]:,}")
print(f"Total volume: ${trades_summary[3]:,.2f}" if trades_summary[3] else "Total volume: N/A")
print(f"Date range: {trades_summary[4]} to {trades_summary[5]}")

## 6. Calculate Backtest P&L

Calculate the profit/loss from copying all trades from top users.

In [ ]:
# Calculate backtest results
backtest_results = con.execute("""
    WITH trade_with_resolution AS (
        SELECT 
            t.proxyWallet,
            t.side,
            t.price as entry_price,
            t.size as original_size,
            t.conditionId,
            t.outcome,
            t.trade_value_usd,
            m.is_resolved,
            m.outcome1_name,
            m.outcome2_name,
            m.outcome1_final_price,
            m.outcome2_final_price,
            CASE 
                WHEN t.outcome = m.outcome1_name THEN m.outcome1_final_price
                WHEN t.outcome = m.outcome2_name THEN m.outcome2_final_price
                ELSE NULL
            END as final_price
        FROM v_trades t
        INNER JOIN top_users u ON t.proxyWallet = u.proxyWallet
        LEFT JOIN market_resolution m ON t.conditionId = m.conditionId
    )
    SELECT 
        COUNT(*) as total_trades,
        COUNT(CASE WHEN is_resolved THEN 1 END) as resolved_trades,
        COUNT(CASE WHEN is_resolved AND 
            CASE 
                WHEN side = 'BUY' THEN (final_price - entry_price) * original_size
                WHEN side = 'SELL' THEN (entry_price - final_price) * original_size
                ELSE 0
            END > 0 THEN 1 END) as winning_trades,
        COUNT(CASE WHEN is_resolved AND 
            CASE 
                WHEN side = 'BUY' THEN (final_price - entry_price) * original_size
                WHEN side = 'SELL' THEN (entry_price - final_price) * original_size
                ELSE 0
            END < 0 THEN 1 END) as losing_trades,
        SUM(CASE WHEN is_resolved THEN trade_value_usd ELSE 0 END) as capital_deployed,
        SUM(CASE WHEN is_resolved THEN
            CASE 
                WHEN side = 'BUY' THEN (final_price - entry_price) * original_size
                WHEN side = 'SELL' THEN (entry_price - final_price) * original_size
                ELSE 0
            END
        ELSE 0 END) as total_pnl,
        AVG(CASE WHEN is_resolved THEN
            CASE 
                WHEN side = 'BUY' THEN (final_price - entry_price) * original_size
                WHEN side = 'SELL' THEN (entry_price - final_price) * original_size
                ELSE NULL
            END
        END) as avg_pnl_per_trade
    FROM trade_with_resolution
""").fetchone()

total_trades, resolved_trades, winning, losing, capital, total_pnl, avg_pnl = backtest_results

print("=" * 50)
print(f"BACKTEST RESULTS (Copy Top {TOP_N} Strategy)")
print("=" * 50)
print(f"Total Trades Copied:    {total_trades:,}")
print(f"Resolved Trades:        {resolved_trades:,}")
print(f"Winning Trades:         {winning:,}")
print(f"Losing Trades:          {losing:,}")
if resolved_trades and resolved_trades > 0:
    win_rate = 100.0 * winning / resolved_trades
    print(f"Win Rate:               {win_rate:.1f}%")
if capital:
    print(f"Capital Deployed:       ${capital:,.2f}")
if total_pnl is not None:
    print(f"Total P&L:              ${total_pnl:,.2f}")
if avg_pnl is not None:
    print(f"Avg P&L per Trade:      ${avg_pnl:.4f}")
if capital and total_pnl and capital > 0:
    roi = 100.0 * total_pnl / capital
    print(f"ROI:                    {roi:.2f}%")

## 7. Best Users to Copy

Find which users would have been most profitable to copy.

In [ ]:
# Best users to copy
best_users_df = con.execute("""
    WITH trade_pnl AS (
        SELECT 
            t.proxyWallet,
            u.userName,
            t.side,
            t.price as entry_price,
            t.size,
            t.outcome,
            t.trade_value_usd,
            m.is_resolved,
            CASE 
                WHEN t.outcome = m.outcome1_name THEN m.outcome1_final_price
                WHEN t.outcome = m.outcome2_name THEN m.outcome2_final_price
                ELSE NULL
            END as final_price
        FROM v_trades t
        INNER JOIN top_users u ON t.proxyWallet = u.proxyWallet
        LEFT JOIN market_resolution m ON t.conditionId = m.conditionId
    ),
    user_stats AS (
        SELECT 
            proxyWallet,
            userName,
            COUNT(*) as total_trades,
            COUNT(CASE WHEN is_resolved THEN 1 END) as resolved_trades,
            SUM(CASE WHEN is_resolved THEN
                CASE 
                    WHEN side = 'BUY' THEN (final_price - entry_price) * size
                    WHEN side = 'SELL' THEN (entry_price - final_price) * size
                    ELSE 0
                END
            ELSE 0 END) as copy_pnl,
            COUNT(CASE WHEN is_resolved AND 
                CASE 
                    WHEN side = 'BUY' THEN (final_price - entry_price) > 0
                    WHEN side = 'SELL' THEN (entry_price - final_price) > 0
                    ELSE false
                END THEN 1 END) as wins
        FROM trade_pnl
        GROUP BY proxyWallet, userName
    )
    SELECT 
        COALESCE(userName, LEFT(proxyWallet, 12) || '...') as user,
        total_trades,
        resolved_trades as resolved,
        wins,
        ROUND(100.0 * wins / NULLIF(resolved_trades, 0), 1) as win_pct,
        ROUND(copy_pnl, 2) as copy_pnl
    FROM user_stats
    WHERE resolved_trades >= 5
    ORDER BY copy_pnl DESC
    LIMIT 20
""").fetchdf()

print("Top 20 Users to Copy (by P&L):")
best_users_df

## 8. Category Performance

Analyze which categories performed best.

In [ ]:
# Category performance
category_df = con.execute("""
    WITH trade_pnl AS (
        SELECT 
            m.category,
            t.side,
            t.price as entry_price,
            t.size,
            t.outcome,
            m.is_resolved,
            CASE 
                WHEN t.outcome = m.outcome1_name THEN m.outcome1_final_price
                WHEN t.outcome = m.outcome2_name THEN m.outcome2_final_price
                ELSE NULL
            END as final_price
        FROM v_trades t
        INNER JOIN top_users u ON t.proxyWallet = u.proxyWallet
        LEFT JOIN market_resolution m ON t.conditionId = m.conditionId
        WHERE m.category IS NOT NULL
    )
    SELECT 
        category,
        COUNT(*) as trades,
        COUNT(CASE WHEN is_resolved THEN 1 END) as resolved,
        ROUND(SUM(CASE WHEN is_resolved THEN
            CASE 
                WHEN side = 'BUY' THEN (final_price - entry_price) * size
                WHEN side = 'SELL' THEN (entry_price - final_price) * size
                ELSE 0
            END
        ELSE 0 END), 2) as total_pnl
    FROM trade_pnl
    GROUP BY category
    ORDER BY total_pnl DESC
""").fetchdf()

print("Performance by Category:")
category_df

## 9. Monthly Performance Data

Calculate monthly P&L for time series analysis.

In [ ]:
# Get monthly P&L
monthly_df = con.execute("""
    WITH trade_pnl AS (
        SELECT 
            DATE_TRUNC('month', to_timestamp(t.timestamp)) as trade_month,
            t.side,
            t.price as entry_price,
            t.size,
            t.outcome,
            t.trade_value_usd,
            m.is_resolved,
            CASE 
                WHEN t.outcome = m.outcome1_name THEN m.outcome1_final_price
                WHEN t.outcome = m.outcome2_name THEN m.outcome2_final_price
                ELSE NULL
            END as final_price
        FROM v_trades t
        INNER JOIN top_users u ON t.proxyWallet = u.proxyWallet
        LEFT JOIN market_resolution m ON t.conditionId = m.conditionId
    )
    SELECT 
        trade_month,
        COUNT(*) as trades,
        COUNT(CASE WHEN is_resolved THEN 1 END) as resolved,
        COUNT(CASE WHEN is_resolved AND 
            CASE WHEN side = 'BUY' THEN (final_price - entry_price) > 0
                 WHEN side = 'SELL' THEN (entry_price - final_price) > 0 ELSE false END 
        THEN 1 END) as wins,
        SUM(trade_value_usd) as volume,
        SUM(CASE WHEN is_resolved THEN
            CASE 
                WHEN side = 'BUY' THEN (final_price - entry_price) * size
                WHEN side = 'SELL' THEN (entry_price - final_price) * size
                ELSE 0
            END
        ELSE 0 END) as monthly_pnl
    FROM trade_pnl
    WHERE trade_month IS NOT NULL
    GROUP BY trade_month
    ORDER BY trade_month
""").fetchdf()

# Parse dates and calculate metrics
monthly_df['trade_month'] = pd.to_datetime(monthly_df['trade_month'])
monthly_df['cumulative_pnl'] = monthly_df['monthly_pnl'].cumsum()
monthly_df['win_rate'] = (monthly_df['wins'] / monthly_df['resolved'] * 100).fillna(0)
monthly_df['monthly_roi'] = (monthly_df['monthly_pnl'] / monthly_df['volume'] * 100).fillna(0)
monthly_df['month_label'] = monthly_df['trade_month'].dt.strftime('%b %Y')

print(f"Monthly data from {monthly_df['month_label'].iloc[0]} to {monthly_df['month_label'].iloc[-1]}")
monthly_df.tail(10)

## 10. Visualizations

Create charts to visualize the backtest results.

In [ ]:
# Key metrics for display
total_pnl = monthly_df['monthly_pnl'].sum()
total_volume = monthly_df['volume'].sum()
total_trades = monthly_df['trades'].sum()
total_resolved = monthly_df['resolved'].sum()
total_wins = monthly_df['wins'].sum()
overall_win_rate = total_wins / total_resolved * 100 if total_resolved > 0 else 0

print("=" * 60)
print("OVERALL METRICS")
print("=" * 60)
print(f"Total P&L:           ${total_pnl:,.2f}")
print(f"Total Volume:        ${total_volume:,.2f}")
print(f"Total Trades:        {total_trades:,}")
print(f"Resolved Trades:     {total_resolved:,}")
print(f"Overall Win Rate:    {overall_win_rate:.1f}%")
print(f"ROI:                 {total_pnl/total_volume*100:.2f}%")

# Monthly stats
profitable_months = (monthly_df['monthly_pnl'] > 0).sum()
total_months = len(monthly_df)
best_month = monthly_df.loc[monthly_df['monthly_pnl'].idxmax()]
worst_month = monthly_df.loc[monthly_df['monthly_pnl'].idxmin()]

print(f"\nProfitable Months:   {profitable_months}/{total_months} ({profitable_months/total_months*100:.0f}%)")
print(f"Best Month:          {best_month['month_label']} (${best_month['monthly_pnl']:,.2f})")
print(f"Worst Month:         {worst_month['month_label']} (${worst_month['monthly_pnl']:,.2f})")

In [ ]:
# Create visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle(f'Polymarket Top {TOP_N} User Copy Strategy - Backtest Results', 
             fontsize=16, fontweight='bold', y=0.98)

# 1. Cumulative P&L Over Time
ax1 = axes[0, 0]
ax1.fill_between(monthly_df['trade_month'], 0, monthly_df['cumulative_pnl'], 
                  alpha=0.3, color='green', label='Cumulative P&L')
ax1.plot(monthly_df['trade_month'], monthly_df['cumulative_pnl'], 
         color='green', linewidth=2.5, marker='o', markersize=4)
ax1.axhline(y=0, color='gray', linestyle='--', linewidth=1)
ax1.set_xlabel('Date')
ax1.set_ylabel('Cumulative P&L ($)')
ax1.set_title('Cumulative P&L Over Time', fontsize=13, fontweight='bold')
ax1.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
ax1.legend(loc='upper left')

# Add annotation for final P&L
final_pnl = monthly_df['cumulative_pnl'].iloc[-1]
ax1.annotate(f'${final_pnl/1e6:.1f}M', 
             xy=(monthly_df['trade_month'].iloc[-1], final_pnl),
             xytext=(10, 10), textcoords='offset points',
             fontsize=12, fontweight='bold', color='green')

# 2. Monthly P&L (Bar Chart)
ax2 = axes[0, 1]
colors = ['green' if x > 0 else 'red' for x in monthly_df['monthly_pnl']]
ax2.bar(monthly_df['trade_month'], monthly_df['monthly_pnl'], 
        color=colors, alpha=0.7, width=20)
ax2.axhline(y=0, color='gray', linestyle='-', linewidth=1)
ax2.set_xlabel('Date')
ax2.set_ylabel('Monthly P&L ($)')
ax2.set_title('Monthly P&L (Green=Profit, Red=Loss)', fontsize=13, fontweight='bold')
ax2.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))

# 3. Monthly ROI (P&L / Volume Ratio)
ax3 = axes[1, 0]
colors_roi = ['green' if x > 0 else 'red' for x in monthly_df['monthly_roi']]
ax3.bar(monthly_df['trade_month'], monthly_df['monthly_roi'], 
        color=colors_roi, alpha=0.7, width=20)
ax3.axhline(y=0, color='gray', linestyle='-', linewidth=1)
avg_roi = monthly_df['monthly_roi'].mean()
ax3.axhline(y=avg_roi, color='blue', linestyle='--', linewidth=2, 
            label=f'Avg ROI ({avg_roi:.1f}%)')
ax3.set_xlabel('Date')
ax3.set_ylabel('Monthly ROI (%)')
ax3.set_title('Monthly Return on Capital (P&L / Volume)', fontsize=13, fontweight='bold')
ax3.legend(loc='upper left')

# 4. Win Rate Over Time
ax4 = axes[1, 1]
ax4.plot(monthly_df['trade_month'], monthly_df['win_rate'], 
         color='purple', linewidth=2, marker='D', markersize=4)
ax4.axhline(y=50, color='red', linestyle='--', linewidth=1, label='50% (Breakeven)')
ax4.axhline(y=overall_win_rate, color='green', linestyle='--', linewidth=1, 
            label=f'Avg ({overall_win_rate:.1f}%)')
ax4.fill_between(monthly_df['trade_month'], 50, monthly_df['win_rate'], 
                  where=monthly_df['win_rate'] > 50, alpha=0.3, color='green')
ax4.fill_between(monthly_df['trade_month'], 50, monthly_df['win_rate'], 
                  where=monthly_df['win_rate'] <= 50, alpha=0.3, color='red')
ax4.set_xlabel('Date')
ax4.set_ylabel('Win Rate (%)')
ax4.set_title('Monthly Win Rate', fontsize=13, fontweight='bold')
ax4.set_ylim(0, 100)
ax4.legend(loc='lower right')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

## 11. Interpretation

### Key Findings

1. **Overall Profitability**: The copy strategy generated significant profits over 3 years
2. **Win Rate ~53%**: Only slightly above breakeven - the edge comes from better position sizing on winners
3. **Monthly Consistency**: ~76% of months were profitable
4. **Growth Trajectory**: Activity exploded in 2024-2025 with 100x volume increase
5. **Risk**: November 2024 showed significant drawdown during election volatility

### Practical Insights

- The strategy works but requires significant capital
- Best applied during high-volume periods (elections, major events)
- Consider filtering to only the top 50-100 users for better ROI
- Real execution would face slippage and timing delays
- Past performance doesn't guarantee future results

In [ ]:
# Close DuckDB connection
con.close()
print("Analysis complete!")